In [6]:
import sys
import json
from pathlib import Path
sys.path.append('/code/src')
from helper import get_segments, df_ann_test, sr_num_frames_test, SegmentDataset

import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

In [5]:
df_ann_test

,video,label,s1,e1,s2,e2,raw_label,raw_rel_video_path,rel_video_path
0,Abuse028_x264.mp4,Abuse,165,240,-1,-1,Abuse,Abuse/Abuse028_x264.mp4,Abuse/Abuse028_x264.mp4
1,Abuse030_x264.mp4,Abuse,1275,1360,-1,-1,Abuse,Abuse/Abuse030_x264.mp4,Abuse/Abuse030_x264.mp4
2,Arrest001_x264.mp4,Arrest,1185,1485,-1,-1,Arrest,Arrest/Arrest001_x264.mp4,Arrest/Arrest001_x264.mp4
3,Arrest007_x264.mp4,Arrest,1530,2160,-1,-1,Arrest,Arrest/Arrest007_x264.mp4,Arrest/Arrest007_x264.mp4
4,Arrest024_x264.mp4,Arrest,1005,3105,-1,-1,Arrest,Arrest/Arrest024_x264.mp4,Arrest/Arrest024_x264.mp4
...,...,...,...,...,...,...,...,...,...
285,Vandalism007_x264.mp4,Vandalism,240,750,-1,-1,Vandalism,Vandalism/Vandalism007_x264.mp4,Vandalism/Vandalism007_x264.mp4
286,Vandalism015_x264.mp4,Vandalism,2010,2700,-1,-1,Vandalism,Vandalism/Vandalism015_x264.mp4,Vandalism/Vandalism015_x264.mp4
287,Vandalism017_x264.mp4,Vandalism,270,330,780,840,Vandalism,Vandalism/Vandalism017_x264.mp4,Vandalism/Vandalism017_x264.mp4
288,Vandalism028_x264.mp4,Vandalism,1830,1980,2400,2670,Vandalism,Vandalism/Vandalism028_x264.mp4,Vandalism/Vandalism028_x264.mp4


In [ ]:
p_rootdir = Path('/code/output/psuedo-captions/gpt-4o/00-rich-context')
# p_scores_dir1 = p_rootdir / 'scored,imagebind,1.0'
# p_scores_dir2 = p_rootdir / 'scored,imagebind,0.8'

In [ ]:
ann_vad = {}
for idx, row in df_ann_test.iterrows():
    num_frames = sr_num_frames_test[row['raw_rel_video_path']]
    bin_label = np.zeros(num_frames, dtype=np.int32)
    bin_label[row['s1']:row['e1']] = 1
    if row['s2'] != -1:
        bin_label[row['s2']:row['e2']] = 1
    ann_vad[row['rel_video_path']] = bin_label

preds_per_scale = []
gaussian_kernel = np.array([.1, .2, .4, .2, .1])
for scale in [1.0, 0.9, 0.8, 0.7]:
    p_scores_dir = p_rootdir / f'scored,imagebind,{scale:.1f}'
    preds = {}
    for rel_video_path, bin_label in ann_vad.items():
        p_json = (p_scores_dir / rel_video_path).with_suffix('.json')
        video_record = json.load(p_json.open())
        segment_records = video_record['segment_records']
        pred = np.zeros(len(bin_label))
        overlap_count = np.zeros(len(bin_label))
        for segment_record in segment_records:
            segment_score = np.mean(segment_record['is_anomalous'])  # anomality of all captions
            pred[segment_record['start_idx']:segment_record['end_idx']+1] += segment_score
            overlap_count[segment_record['start_idx']:segment_record['end_idx']+1] += 1
        overlap_count = np.maximum(overlap_count, 1)
        preds[rel_video_path] = pred / overlap_count
        preds[rel_video_path] = np.convolve(preds[rel_video_path], gaussian_kernel, mode='same')  # gaussian smoothing
    preds_per_scale.append(preds)

    all_preds, all_labels = np.array([]), np.array([])
    for rel_video_path, bin_label in ann_vad.items():
        all_preds = np.concatenate([all_preds, preds[rel_video_path]])
        all_labels = np.concatenate([all_labels, bin_label])
    auc = roc_auc_score(all_labels.astype(int), all_preds)
    print(f'Scale: {scale:.1f} | AUC: {auc:.4f}')

Scale: 1.0 | AUC: 0.6066
Scale: 0.9 | AUC: 0.7559


In [22]:
action_classes = df_ann_test['label'].unique()
for action_class in action_classes:
    all_preds, all_labels = np.array([]), np.array([])
    for rel_video_path, bin_label in ann_vad.items():
        if df_ann_test[df_ann_test['rel_video_path'] == rel_video_path]['label'].values[0] != action_class:
            continue
        all_preds = np.concatenate([all_preds, preds[rel_video_path]])
        all_labels = np.concatenate([all_labels, bin_label])
    if action_class == 'Normal':
        all_preds = np.concatenate([all_preds, [1]])
        all_labels = np.concatenate([all_labels, [1]])
    auc = roc_auc_score(all_labels.astype(int), all_preds)
    print(f'{action_class}: {auc}')

Abuse: 0.409057582260372
Arrest: 0.5687513174132914
Arson: 0.6773560089704245
Assault: 0.8405974488912107
Burglary: 0.6139725512215154
Explosion: 0.58861947402039
Fighting: 0.5436728373797779
Normal: 0.9904531479954692
RoadAccidents: 0.5718458006997202
Robbery: 0.6598400211042914
Shooting: 0.6116256242830598
Shoplifting: 0.6898984487766024
Stealing: 0.693517962735018
Vandalism: 0.6636571180010824
